In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import json
import pycountry
import pandas as pd
    
JSON_PATH = "/Users/seyoung/code/geo-quest-mania/src/lib/countries-110m.json"
COUNTRY_CONTINENT_MAP_PATH = "/Users/seyoung/Downloads/fujitsu_country_continent_code.csv"
df = pd.read_csv(COUNTRY_CONTINENT_MAP_PATH, dtype=str, 
                 keep_default_na=False,   # don't treat "NA", "NaN", etc. as NaN
                 na_filter=False          # disable NA detection completely
                )

## About World Atlas TopoJSON
- Source: https://github.com/topojson/world-atlas
- Visualization: https://observablehq.com/@d3/world-map
- world.objects.countries
- country.id - the three-digit ISO 3166-1 numeric country code, such as "528"
- country.properties.name - the country name, such as "Netherlands"
- world.objects.land

In [2]:
pycountry.countries.get(numeric="528")

Country(alpha_2='NL', alpha_3='NLD', flag='🇳🇱', name='Netherlands', numeric='528', official_name='Kingdom of the Netherlands')

In [3]:
pycountry.countries.get(numeric="528").official_name

'Kingdom of the Netherlands'

In [2]:
import pycountry

def continent_code_by_country_code(country_code: str, df) -> str:
    return df[df["country_code"] == country_code].iloc[0].continent_code
    # return pc.country_alpha2_to_continent_code(country_code)

def country_code_by_name(name: str) -> str:
    """
    Return the ISO 3166-1 alpha-2 country code for a given country name.
    """
    try:
        # lookup() handles exact names, common names, and many variants
        country = pycountry.countries.lookup(name)
        return country.alpha_2
    except LookupError:
        return None

def country_flag_by_numeric(numeric: int) -> str:
    try:
        # lookup() handles exact names, common names, and many variants
        country = pycountry.countries.get(numeric=str(numeric))
        return country.flag
    except LookupError:
        return None
    

def country_code_by_numeric(numeric: int) -> str:
    """
    Return the ISO 3166-1 alpha-2 country code for a given country numeric code.
    """
    try:
        # lookup() handles exact names, common names, and many variants
        country = pycountry.countries.get(numeric=str(numeric))
        return country.alpha_2
    except LookupError:
        return None

def country_code(query) -> str:
    if isinstance(query, str):
        return country_code_by_name(query)
    elif isinstance(query, int):
        return country_code_by_numeric(query)
    else:
        raise TypeError(
            f"query must be of type str or int, not {type(query).__name__}"
        )

def country_official_by_numeric(numeric: str) -> str:
    """
    Return the ISO 3166-1 alpha-2 country code for a given country name.
    """
    try:
        # lookup() handles exact names, common names, and many variants
        country = pycountry.countries.get(numeric=str(numeric))
        return country.official_name
    except LookupError:
        return None


# Examples
print(country_code("Finland"))        # → 'FI'
print(country_code("Korea"))          # → 'KR'  (Republic of Korea)
print(country_code("South Korea"))    # → 'KR'
print(country_code("North Korea"))    # → 'KP'
print(country_flag_by_numeric("528"))

FI
None
KR
KP
🇳🇱


In [3]:
print(continent_code_by_country_code(country_code("Antarctica"), df))

AQ


In [4]:
country_code("Namibia")

'NA'

## Load TopoJSON

In [5]:
with open(JSON_PATH) as f:
    topo = json.load(f)

countries = topo['objects']['countries']

In [4]:
topo.keys()

dict_keys(['type', 'objects', 'arcs', 'bbox', 'transform'])

In [6]:
topo['objects'].keys()

dict_keys(['countries', 'land'])

In [10]:
topo['objects']['countries'].keys()

dict_keys(['type', 'geometries'])

In [10]:
countries.keys()

dict_keys(['type', 'geometries'])

In [12]:
len(countries['geometries'])

177

In [14]:
len(topo_worldpopulationreview['features'])

175

In [31]:
countries["geometries"][0]

{'type': 'MultiPolygon',
 'arcs': [[[0]], [[1]]],
 'id': '242',
 'properties': {'name': 'Fiji'}}

## Add data to JSON

In [13]:
# Sort countries alphabetically
countries['geometries'].sort(key=lambda g: g["properties"]["name"])

for i, c in enumerate(countries['geometries']):
    name = c['properties']['name']
    try:
        c_numeric = c['id']
    except KeyError:
        pass
    else:
        code = country_code(c_numeric)
        c['properties']['code'] = code
        flag = country_flag_by_numeric(c_numeric)
        c['properties']['flag'] = flag
        continent_code = continent_code_by_country_code(code, df)
        c['properties']['continent_code'] = continent_code
        
        # print(f"'[{i+1}] {flag} id: '{country_code(c_numeric)}' {name}': {continent_code}")
        print("{ "+ f"id: \"{country_code(c_numeric)}\", name: \"{name}\", continent_code: \"{continent_code}\"" + " },")
         # flag='🇳🇱', name='Netherlands', numeric='528', official_name='Kingdom of the Netherlands')
    

{ id: "AF", name: "Afghanistan", continent_code: "AS" },
{ id: "AL", name: "Albania", continent_code: "EU" },
{ id: "DZ", name: "Algeria", continent_code: "AF" },
{ id: "AO", name: "Angola", continent_code: "AF" },
{ id: "AQ", name: "Antarctica", continent_code: "AQ" },
{ id: "AR", name: "Argentina", continent_code: "SA" },
{ id: "AM", name: "Armenia", continent_code: "AS" },
{ id: "AU", name: "Australia", continent_code: "OC" },
{ id: "AT", name: "Austria", continent_code: "EU" },
{ id: "AZ", name: "Azerbaijan", continent_code: "AS" },
{ id: "BS", name: "Bahamas", continent_code: "NA" },
{ id: "BD", name: "Bangladesh", continent_code: "AS" },
{ id: "BY", name: "Belarus", continent_code: "EU" },
{ id: "BE", name: "Belgium", continent_code: "EU" },
{ id: "BZ", name: "Belize", continent_code: "NA" },
{ id: "BJ", name: "Benin", continent_code: "AF" },
{ id: "BT", name: "Bhutan", continent_code: "AS" },
{ id: "BO", name: "Bolivia", continent_code: "SA" },
{ id: "BA", name: "Bosnia and Herz

In [16]:
for i, c in enumerate(countries['geometries']):
    print(c['properties']['name'])

Afghanistan
Albania
Algeria
Angola
Antarctica
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bangladesh
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herz.
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Central African Rep.
Chad
Chile
China
Colombia
Congo
Costa Rica
Croatia
Cuba
Cyprus
Czechia
Côte d'Ivoire
Dem. Rep. Congo
Denmark
Djibouti
Dominican Rep.
Ecuador
Egypt
El Salvador
Eq. Guinea
Eritrea
Estonia
Ethiopia
Falkland Is.
Fiji
Finland
Fr. S. Antarctic Lands
France
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Greenland
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Lithuania
Luxembourg
Macedonia
Madagascar
Malawi
Malaysia
Mali
Mauritania
Mexico
Moldova
Mongolia
Montenegro
Morocco
Mozambique
Myanmar
N. Cyprus
Namibia
Nepal
Netherlands
New Caledonia
New Zealand
Nicara

In [17]:
topo['objects']['countries']['geometries'][0]

{'type': 'Polygon',
 'arcs': [[-69, 408, 409, -407, 410, 411]],
 'id': '004',
 'properties': {'name': 'Afghanistan', 'code': 'AF', 'flag': '🇦🇫'}}

## Write JSON file

In [8]:
with open("/Users/seyoung/code/geo-quest-mania/src/lib/countries-110m-with-country-code.json", "w", encoding="utf-8") as f:
    # 3. Serialize `data` to JSON and write it into the file:
    json.dump(topo, f, ensure_ascii=False, indent=4)

In [10]:
topo['objects']['land']

{'type': 'GeometryCollection',
 'geometries': [{'type': 'MultiPolygon',
   'arcs': [[[0]],
    [[1]],
    [[3,
      320,
      184,
      255,
      323,
      104,
      322,
      311,
      313,
      315,
      289,
      284,
      273,
      290,
      293,
      297,
      305,
      307,
      302,
      304,
      263,
      336,
      258,
      272,
      13,
      573,
      341,
      338,
      577,
      575,
      329,
      332,
      423,
      487,
      535,
      153,
      435,
      164,
      436,
      461,
      477,
      489,
      484,
      490,
      592,
      497,
      548,
      525,
      240,
      507,
      505,
      506,
      242,
      501,
      503,
      474,
      528,
      476,
      446,
      142,
      464,
      466,
      468,
      158,
      549,
      430,
      172,
      161,
      387,
      391,
      389,
      522,
      385,
      371,
      372,
      543,
      374,
      380,
      401,
      394,
      404,
      422,